In [70]:
import pandas as pd
import numpy as np

# Read in eviction data
evict = pd.read_csv('~/Desktop/project1030/SF/Eviction_Notices.csv')
print(evict.columns)


# Restrict to subset of columns 
evict = evict[['Eviction ID', 'File Date', 'Non Payment', 'Illegal Use', \
               'Nuisance', 'Neighborhoods - Analysis Boundaries', 'Location']]

print(evict.shape)
print(evict.head())

Index(['Eviction ID', 'Address', 'City', 'State',
       'Eviction Notice Source Zipcode', 'File Date', 'Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development',
       'Good Samaritan Ends', 'Constraints Date', 'Supervisor District',
       'Neighborhoods - Analysis Boundaries', 'Location'],
      dtype='object')
(38117, 7)
  Eviction ID   File Date  Non Payment  Illegal Use  Nuisance  \
0     M171880  07/27/2017        False        False      True   
1     M171925  07/27/2017        False        False     False   
2     M171926  07/27/2017        False        False     False   
3     M171834  07/25/2017        False        False      True   
4     M141352  06/10/2014         True     

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
from datetime import datetime
from dateutil.parser import parse

# Convert string dates to datetime format
evict['date'] = [datetime.strptime(x, '%m/%d/%Y') for x in evict['File Date'].values]
evict.sort_values(by='date', inplace=True)

In [72]:
# Restrict to evictions in time range Jan 2003 to May 2015
data_start = '2003-01-01'
data_end = '2015-05-30'

evict_new = evict[(evict['date'] >= datetime.strptime(data_start, '%Y-%m-%d')) & \
                  (evict['date'] <= datetime.strptime(data_end, '%Y-%m-%d'))]

# Drop rows with NA values for location
evict_new = evict_new.dropna(axis=0, how='any')

evict_new.sort_values(by='date', inplace=True)

In [73]:
import re

# Write function to convert "(x, y)" to x and y columns 
def convert_coords(coords):
    m = re.split(',', coords)
    x_trim = m[0].replace("(", '')
    x_num = float(x_trim)

    y_trim = m[1].replace(")", '')
    y_num = float(y_trim)
    return (x_num, y_num)

x_coords = [i[1] for i in list(map(convert_coords, evict_new['Location']))]
y_coords = [i[0] for i in list(map(convert_coords, evict_new['Location']))]

evict_new['X'] = x_coords
evict_new['Y'] = y_coords

In [74]:
import geopandas as gpd
import os
from geopandas import GeoDataFrame
import shapely.wkt

# Read in csv file containing SF census tracts and corresponding MULTIPOLYGON objects
data_path = '/Users/Sam/Desktop/project1030/SF'
census_tr = pd.read_csv(os.path.join(data_path, 'Census_2010_Tracts.csv'))

geometry = census_tr['the_geom'].map(shapely.wkt.loads)
print(census_tr.head())

crs = {'init': 'epsg:4326'}

# Restrict to mainland SF 
census_tr = census_tr[census_tr['INTPTLON10'] > -122.6]

sf_census_tracts = GeoDataFrame(census_tr, crs=crs, geometry=geometry)

   STATEFP10  TRACTCE10                                           the_geom  \
0          6      16500  MULTIPOLYGON (((-122.446471 37.775802, -122.44...   
1          6      16400  MULTIPOLYGON (((-122.44033999999999 37.7765799...   
2          6      16300  MULTIPOLYGON (((-122.429152 37.778006999999995...   
3          6      16100  MULTIPOLYGON (((-122.428909 37.778039, -122.42...   
4          6      16000  MULTIPOLYGON (((-122.420425 37.780583, -122.42...   

   COUNTYFP10     GEOID10  NAME10        NAMELSAD10 MTFCC10 FUNCSTAT10  \
0          75  6075016500   165.0  Census Tract 165   G5020          S   
1          75  6075016400   164.0  Census Tract 164   G5020          S   
2          75  6075016300   163.0  Census Tract 163   G5020          S   
3          75  6075016100   161.0  Census Tract 161   G5020          S   
4          75  6075016000   160.0  Census Tract 160   G5020          S   

   ALAND10  AWATER10  INTPTLAT10  INTPTLON10  
0   370459         0   37.774196 -122.4

In [75]:
from shapely.geometry import Point

# First convert all eviction to Point objects
evict_locs = [Point(xy) for xy in zip(evict_new['X'], evict_new['Y'])]
evict_locs_df = GeoDataFrame(evict_locs, crs=crs, geometry=evict_locs)

evict_new['locs'] = evict_locs

In [76]:
# Spatial join the census tracts polygons and the eviction points
evict_census = gpd.sjoin(evict_locs_df, sf_census_tracts, how="inner", op='within')

In [77]:
print(evict_census.columns)

# NAME10 is the column from the census tracts dataset with the numeric identifier
# How many evictions happened in each census tract?
#print(evict_census['NAME10'].value_counts())

Index([            0,    'geometry', 'index_right',   'STATEFP10',
         'TRACTCE10',    'the_geom',  'COUNTYFP10',     'GEOID10',
            'NAME10',  'NAMELSAD10',     'MTFCC10',  'FUNCSTAT10',
           'ALAND10',    'AWATER10',  'INTPTLAT10',  'INTPTLON10'],
      dtype='object')


In [78]:
# Extract X and Y coordinates
x_coords = evict_census['geometry'].apply(lambda p: p.x)
y_coords = evict_census['geometry'].apply(lambda p: p.y)

# Create new truncated df containing only Point object and census tract name
evict_census_trunc = evict_census[['NAME10', 'the_geom']]
evict_census_trunc['X'] = x_coords
evict_census_trunc['Y'] = y_coords

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [79]:
print(evict_census_trunc.shape)
print(evict_new.shape)

# Remove duplicates from spatially merged dataset (evict_census_trunc)
evict_census_trunc.drop_duplicates(inplace=True)
print(evict_census_trunc.shape)

# Now, merge on census tract information to evict_new dataset, joining on the coordinate variables
evict_merged = pd.merge(evict_new, evict_census_trunc, how='inner', on=['X', 'Y'])

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


(18614, 4)
(18614, 11)
(6283, 4)


In [80]:
print(evict_merged.columns)
print(evict_merged.shape)
evict_merged.rename(columns= {'NAME10': 'census_tr', 'the_geom': 'census_tr_poly'}, inplace=True)

Index(['Eviction ID', 'File Date', 'Non Payment', 'Illegal Use', 'Nuisance',
       'Neighborhoods - Analysis Boundaries', 'Location', 'date', 'X', 'Y',
       'locs', 'NAME10', 'the_geom'],
      dtype='object')
(18614, 13)


In [81]:
# Restrict to important columns
evict_merged = evict_merged[['date', 'Non Payment', 'Illegal Use', 'X', 'Y', 'census_tr']]

In [82]:
evict_merged.sort_values(by='date', inplace=True)

In [83]:
print(evict_merged.head(20))

          date  Non Payment  Illegal Use           X          Y  census_tr
0   2003-01-02        False        False -122.437094  37.779627     158.02
1   2003-01-02        False        False -122.415634  37.758035     228.01
16  2003-01-02        False        False -122.469258  37.761105     302.02
11  2003-01-02        False        False -122.388525  37.728779     232.00
55  2003-01-03        False        False -122.442475  37.760487     204.01
56  2003-01-03        False        False -122.442475  37.760487     204.01
63  2003-01-03        False        False -122.418714  37.785009     122.02
93  2003-01-03        False        False -122.481779  37.776369     477.02
95  2003-01-03        False        False -122.455064  37.780743     156.00
96  2003-01-03        False        False -122.452280  37.731433     311.00
54  2003-01-03        False        False -122.442475  37.760487     204.01
52  2003-01-03        False        False -122.430701  37.734447     218.00
101 2003-01-06        Fal

In [84]:
# Create new df with total date range from Jan 2003 to May 2015
all_dates = pd.Series(pd.date_range(start = data_start, end = data_end))

In [85]:
evict_all_dates = pd.DataFrame(all_dates, columns=['date'])

(4533, 1)


In [ ]:
import itertools

# Create dataframe equal to cartesian product of all_dates and census_tracts


def cartesian(df1, df2):
    rows = itertools.product(df1.iterrows(), df2.iterrows())

    df = pd.DataFrame(left.append(right) for (_, left), (_, right) in rows)
    return df.reset_index(drop=True)

In [86]:
# Test case: can we get the number of days between two datetime stamps?
delta = evict_all_dates['date'].values[20] - evict_all_dates['date'].values[1]
days = delta.astype('timedelta64[D]')
days / np.timedelta64(1, 'D')

19.0

In [87]:
# Function to calculate difference between two datetimes: datetime_2 (later) and datetime_init (earlier)
def calc_time_delta(datetime_2, datetime_init):
    delta = datetime_2 - datetime_init
    days = delta.astype('timedelta64[D]')
    return (days / np.timedelta64(1, 'D'))

# Map this function over the entire datetime column to get minutes elapsed since beginning
# of dataset
evict_merged['days_elapsed'] = np.apply_along_axis(func1d = calc_time_delta, axis = 0, \
                              arr=evict_merged['date'].values, datetime_init=evict_merged['date'].values[0])

In [88]:
print(evict_merged.head())

         date  Non Payment  Illegal Use           X          Y  census_tr  \
0  2003-01-02        False        False -122.437094  37.779627     158.02   
1  2003-01-02        False        False -122.415634  37.758035     228.01   
16 2003-01-02        False        False -122.469258  37.761105     302.02   
11 2003-01-02        False        False -122.388525  37.728779     232.00   
55 2003-01-03        False        False -122.442475  37.760487     204.01   

    days_elapsed  
0            0.0  
1            0.0  
16           0.0  
11           0.0  
55           1.0  


In [89]:
# Convert non-payment and illegal use to numeric
evict_merged['non_payment'] = evict_merged['Non Payment'].astype(int)
evict_merged['illegal_use'] = evict_merged['Illegal Use'].astype(int)
evict_merged['eviction'] = 1

evict_merged.drop(columns=['Non Payment', 'Illegal Use'], inplace=True)

In [95]:
print(evict_merged.head())
print(evict_merged.tail())

         date           X          Y  census_tr  days_elapsed  non_payment  \
0  2003-01-02 -122.437094  37.779627     158.02           0.0            0   
1  2003-01-02 -122.415634  37.758035     228.01           0.0            0   
16 2003-01-02 -122.469258  37.761105     302.02           0.0            0   
11 2003-01-02 -122.388525  37.728779     232.00           0.0            0   
55 2003-01-03 -122.442475  37.760487     204.01           1.0            0   

    illegal_use  eviction  
0             0         1  
1             0         1  
16            0         1  
11            0         1  
55            0         1  
            date           X          Y  census_tr  days_elapsed  non_payment  \
18612 2015-05-29 -122.389871  37.721036     234.00        4530.0            0   
18091 2015-05-29 -122.401167  37.730412     230.01        4530.0            0   
7879  2015-05-29 -122.473557  37.742210     308.00        4530.0            0   
14671 2015-05-29 -122.387018  37.733592

In [106]:
from datetime import date
from datetime import datetime

# Convert date to datetime
def date_to_datetime(date):
    #return str(date)
    return pd.to_datetime(str(date))

date_to_datetime(evict_merged['date'].values[0])

Timestamp('2003-01-02 00:00:00')

In [120]:
# Reformat the eviction data so that it matches what we have in the crime dataset (pad with zeros)
evict_merged['datetime'] = evict_merged['date'].apply(lambda x: date_to_datetime(x))
evict_merged['time'] = evict_merged['datetime'].apply(lambda x: x.time)
evict_merged['Category'] = 'EVICTION'
evict_merged['DayOfWeek'] = "NULL"
evict_merged['year'] = evict_merged['datetime'].apply(lambda x: x.year)
evict_merged['month'] = evict_merged['datetime'].apply(lambda x: x.month)

In [122]:
#evict_merged.drop(columns=['DayofWeek'], inplace=True)

In [124]:
evict_export = evict_merged.drop(columns=['X', 'Y', 'days_elapsed'])
print(evict_export.columns)

Index(['date', 'census_tr', 'non_payment', 'illegal_use', 'eviction',
       'datetime', 'time', 'Category', 'year', 'month', 'DayOfWeek'],
      dtype='object')
(18614, 11)


In [125]:
# Export eviction data to be integrated with crime data
evict_export.to_csv('~/Desktop/project1030/SF/evict_processed.csv', index=False)